# **Data Cleaning**

## Import Packages

In [49]:
# Import packages

from bs4 import BeautifulSoup as bs4
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import scipy
import math

## Assign driver

In [4]:
driver = webdriver.Chrome() # Driver dibuka di chrome

min_page = 10 # Set minimal page

## Testing getAttr

In [5]:
url = 'https://www.tokopedia.com/search?navsource=&page=1&q=seblak&srp_componeqnt_id=02.01.00.00&srp_page_id=&srp_page_title=&st='
driver.get(url)

html = driver.page_source

# parsing (uraikan)
soup = bs4(html, 'html.parser')

print(soup.find('div', {'class' : 'prd_link-product-price css-h66vau'}).get_text()) # find div harga terdekat

Rp51.152


## Web Scrap Tokopedia

1. Membuat list untuk id, produk, harga, seller, kota, jumlah terjual, dan rating produk.
2. Mengambil setiap page sampai minimal page yang di-atur, lalu webdriver meng-scroll sampai kebawah agar semua elemen dimuat.
2. Memuat web yang masih berantakan dengan page_source, dan diuraikan dengan bs4(html, 'parser')
3. For loop untuk setiap row dan box product untuk mengambil informasi produk di setiap box
4. Membuat identifier dengan format P{nomer halaman}-R{urutan row}-B{urutan box} dan dimasukkan ke list id
5. Mengambil elemen produk, harga, seller, kota, jumlah terjual, dan rating produk dari box produk, dan dimasukkan ke masing - masing list.
6. Membuat dataframe dengan semua list menjadi komponennya.

In [6]:
id = []         # Melihat page ke-berapa
product = []    # Nama produk
price = []      # Harga produk
seller = []     # Penjual
city = []       # Kota penjual
sellcount = []  # Banyaknya terjual
rating = []     # Rating produk

min_page = 15 # Minimal page

for page in range(1, min_page + 1):
    
    # Load url untuk tiap page
    url = f'https://www.tokopedia.com/search?navsource=&page={page}&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st='
    driver.get(url)
    
    # Webdriver scroller
    y = 500
    for timer in range(0,17):
        driver.execute_script("window.scrollTo(0, "+str(y)+")")
        y += 500  
        time.sleep(1)
        
    html = driver.page_source
    soup = bs4(html, 'html.parser') # parsing (uraikan)
    
    search_area = soup.find('div', {'data-testid' : 'divSRPContentProducts'})
    rows = search_area.find_all('div', {'class' : 'css-jza1fo'}) # For loop untuk setiap row
    
    unloaded = search_area.find_all('div', {'class' : 'position:relative'})
    # print(f"Number of rows: {len(rows)}")
    for row in rows:
        boxes = row.find_all('div', {'class' : 'pcv3__container css-1izdl9e'})
        # print(f"Number of boxes : {len(boxes)}")
        
        for infobox in boxes:
            
            # infobox = box.find('a', {'class' : 'pcv3__info-content'})
            
            id.append(f"P{page}-R{rows.index(row) + 1}-B{boxes.index(infobox) + 1}") # identifier berdasarkan page, row & box
            
            # Kolom nama
            try:
                name = infobox.find('div', {'class' : 'prd_link-product-name css-3um8ox'})
                product.append(name.get_text().replace(', ', ' - ').replace(',', ' - '))
            except:
                product.append(None)
            
            # Kolom harga
            try:
                prc  = infobox.find('div', {'class' : 'prd_link-product-price css-h66vau'})
                price.append(float(prc.get_text().replace('Rp', '').replace('.', '')))
            except:
                price.append(None)
            
            # Kolom penjual
            try:
                ppl  = infobox.find('span', {'class' : 'prd_link-shop-name css-1kdc32b flip'})
                seller.append(ppl.get_text())
            except:
                seller.append('Dilayani Tokopedia')
            
            # Kolom kota penjual
            try:
                loc  = infobox.find('span', {'class' : 'prd_link-shop-loc'})
                city.append(loc.get_text())
            except:
                city.append(None)
            
            # Kolom kota penjual
            try:
                sc   = infobox.find('span', {'class' : 'prd_label-integrity'})
                sellcount.append(int(sc.get_text().replace(' terjual', '').replace('+', '').replace('rb', '000')))
            except:
                sellcount.append(None)
            
            # Kolom rating produk
            try:
                rate = infobox.find('span', {'class' : 'prd_rating-average-text'})
                rating.append(float(rate.get_text()))
            except:
                rating.append(None)
                
    # Print console page selesai di scrap, dengan menampilkan length setiap list yang di-scrap
    print(f"Page done | Page: {page} | Produk : {len(product)} | Price : {len(price)} | Seller : {len(seller)} | City : {len(city)} | Sell Count : {len(sellcount)} | Rating : {len(rating)}") 
    

df = pd.DataFrame()

df['ID'] = id
df['Product'] = product
df['Price'] = price
df['Seller'] = seller
df['City'] = city
df['Sell Count'] = sellcount
df['Rating'] = rating

Page done | Page: 1 | Produk : 85 | Price : 85 | Seller : 85 | City : 85 | Sell Count : 85 | Rating : 85
Page done | Page: 2 | Produk : 170 | Price : 170 | Seller : 170 | City : 170 | Sell Count : 170 | Rating : 170
Page done | Page: 3 | Produk : 255 | Price : 255 | Seller : 255 | City : 255 | Sell Count : 255 | Rating : 255
Page done | Page: 4 | Produk : 340 | Price : 340 | Seller : 340 | City : 340 | Sell Count : 340 | Rating : 340
Page done | Page: 5 | Produk : 425 | Price : 425 | Seller : 425 | City : 425 | Sell Count : 425 | Rating : 425
Page done | Page: 6 | Produk : 488 | Price : 488 | Seller : 488 | City : 488 | Sell Count : 488 | Rating : 488
Page done | Page: 7 | Produk : 573 | Price : 573 | Seller : 573 | City : 573 | Sell Count : 573 | Rating : 573
Page done | Page: 8 | Produk : 653 | Price : 653 | Seller : 653 | City : 653 | Sell Count : 653 | Rating : 653
Page done | Page: 9 | Produk : 738 | Price : 738 | Seller : 738 | City : 738 | Sell Count : 738 | Rating : 738
Page do

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1129 entries, 0 to 1247
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          1129 non-null   object 
 1   Product     1129 non-null   object 
 2   Price       1129 non-null   float64
 3   Seller      1129 non-null   object 
 4   City        1129 non-null   object 
 5   Sell Count  1129 non-null   float64
 6   Rating      1129 non-null   float64
dtypes: float64(3), object(4)
memory usage: 70.6+ KB


In [21]:
df.head(20)

,ID,Product,Price,Seller,City,Sell Count,Rating
0,P1-R1-B1,Kylafood Seblak Original,21375.0,kylafood,Bandung,10000.0,4.9
1,P1-R1-B2,Seblak Rafael/Seblak Coet Instan Halal,26600.0,Foodstocks,Jakarta Selatan,100.0,5.0
2,P1-R1-B3,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,17000.0,Central Seblak Nusantara,Tangerang Selatan,2000.0,4.9
3,P1-R1-B4,KERUPUK SEBLAK MENTAH 1KG WARNA WARNI - Mawar ...,26500.0,BociKakang,Jakarta Selatan,250.0,5.0
4,P1-R1-B5,KERUPUK MENTAH KERUPUK SEBLAK MENTAH 500 GR - ...,14500.0,BociKakang,Jakarta Selatan,100.0,4.9
5,P1-R1-B6,Seblak Instan Pedas Home Made,3500.0,the Dhecip,Tangerang Selatan,3000.0,4.9
6,P1-R1-B7,SEBLAK INSTANT TERMURAH,4450.0,Pusat Cuankie & Cemilan Frozen,Jakarta Timur,10000.0,4.8
7,P1-R1-B8,Seblak Rafael - Seblak Coet Instan Halal,25000.0,Brother Meat Shop,Depok,500.0,5.0
8,P1-R1-B9,Prabu Snack Kerupuk Seblak Snack 150 gram,9500.0,Dilayani Tokopedia,Jakarta Barat,100.0,4.9
9,P1-R1-B10,Kylafood Seblak Rempah Autentik,12500.0,Kylafood Jakarta,Jakarta Selatan,100.0,5.0


In [22]:
df['City'].unique()

array(['Bandung', 'Jakarta Selatan', 'Tangerang Selatan', 'Jakarta Timur',
       'Depok', 'Jakarta Barat', 'Kab. Tangerang', 'Kab. Bogor',
       'Kab. Bekasi', 'Kab. Bandung', 'Magelang', 'Kab. Garut',
       'Surabaya', 'Jakarta Pusat', 'Tangerang', 'Bekasi', 'Luar kota',
       'Kab. Tasikmalaya', 'Cimahi', 'Tasikmalaya', 'Kab. Sidoarjo',
       'Kab. Majalengka', 'Kab.Ciamis', 'Kab. Indramayu', 'Jakarta Utara',
       'Bogor', 'Surakarta', 'Malang', 'Kab. Cianjur', 'Kab. Klaten',
       'Sukabumi', 'Kab. Bandung Barat', 'Kab. Cirebon', 'Semarang',
       'Kediri', 'Kab. Bantul', 'Banjarbaru', 'Bandar Lampung',
       'Kab. Gresik', 'Makassar', 'Kab. Malang', 'Palembang',
       'Kab. Mojokerto', 'Pekanbaru', 'Kab. Jember', 'Kab. Sukabumi',
       'Kab. Madiun', 'Medan', 'Kab. Sumedang', 'Kab. Kebumen', 'Cirebon',
       'Kab. Boyolali', 'Kab. Purwakarta', 'Kab. Magelang',
       'Kab. Jombang', 'Kab. Tegal', 'Kab. Sleman', 'Banjar', 'Serang',
       'Kab. Kediri', 'Samarinda'], dt

In [23]:
# Mengisi cell null di sellcount dengan 0.0
df['Sell Count'].fillna(0.0)

0       10000.0
1         100.0
2        2000.0
3         250.0
4         100.0
         ...   
1241      250.0
1242        6.0
1244      750.0
1246     1000.0
1247       70.0
Name: Sell Count, Length: 1129, dtype: float64

In [24]:
df.dropna(axis = 0, how = 'any', inplace = True) # Dropna di rating
df.info() # Melihat info column setelah dropna

<class 'pandas.core.frame.DataFrame'>
Index: 1129 entries, 0 to 1247
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          1129 non-null   object 
 1   Product     1129 non-null   object 
 2   Price       1129 non-null   float64
 3   Seller      1129 non-null   object 
 4   City        1129 non-null   object 
 5   Sell Count  1129 non-null   float64
 6   Rating      1129 non-null   float64
dtypes: float64(3), object(4)
memory usage: 70.6+ KB


# **Data Preparation**

In [25]:
df.info() # Melihat info kolom setelah di-cleaning

<class 'pandas.core.frame.DataFrame'>
Index: 1129 entries, 0 to 1247
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          1129 non-null   object 
 1   Product     1129 non-null   object 
 2   Price       1129 non-null   float64
 3   Seller      1129 non-null   object 
 4   City        1129 non-null   object 
 5   Sell Count  1129 non-null   float64
 6   Rating      1129 non-null   float64
dtypes: float64(3), object(4)
memory usage: 70.6+ KB


Karena format dan tipe tiap kolom sudah benar, maka hanya perlu mengubah cell 'Dilayani Tokopedia' di kolom City menjadi 'Luar kota'.

In [26]:
# Data backup

df_backup = df

In [27]:
# np.where untuk if-statement kalau cell = 'Dilayani Tokopedia' maka diganti menjadi 'Luar kota'
df['City'] = np.where(df['City'] == 'Dilayani Tokopedia', 'Luar kota', df['City'])
df['City'].unique()

array(['Bandung', 'Jakarta Selatan', 'Tangerang Selatan', 'Jakarta Timur',
       'Depok', 'Jakarta Barat', 'Kab. Tangerang', 'Kab. Bogor',
       'Kab. Bekasi', 'Kab. Bandung', 'Magelang', 'Kab. Garut',
       'Surabaya', 'Jakarta Pusat', 'Tangerang', 'Bekasi', 'Luar kota',
       'Kab. Tasikmalaya', 'Cimahi', 'Tasikmalaya', 'Kab. Sidoarjo',
       'Kab. Majalengka', 'Kab.Ciamis', 'Kab. Indramayu', 'Jakarta Utara',
       'Bogor', 'Surakarta', 'Malang', 'Kab. Cianjur', 'Kab. Klaten',
       'Sukabumi', 'Kab. Bandung Barat', 'Kab. Cirebon', 'Semarang',
       'Kediri', 'Kab. Bantul', 'Banjarbaru', 'Bandar Lampung',
       'Kab. Gresik', 'Makassar', 'Kab. Malang', 'Palembang',
       'Kab. Mojokerto', 'Pekanbaru', 'Kab. Jember', 'Kab. Sukabumi',
       'Kab. Madiun', 'Medan', 'Kab. Sumedang', 'Kab. Kebumen', 'Cirebon',
       'Kab. Boyolali', 'Kab. Purwakarta', 'Kab. Magelang',
       'Kab. Jombang', 'Kab. Tegal', 'Kab. Sleman', 'Banjar', 'Serang',
       'Kab. Kediri', 'Samarinda'], dt

# **Business Understanding/Problem Statement**

## Metode SMART

**S | Specific**
* Berhasil menjual produk seblak di Tokopedia

**M | Measurable**
* Jumlah terjual produk seller >= Jumlah terjual 50% populasi

**A | Achievable**
* Harga jual produk berada dalam range rata - rata harga produk, dan memiliki harga yang masuk akal

**R | Relevant**
* Semakin tinggi seblak terbeli, maka semakin tinggi pula profit yang diterima.

**T | Timebound Goals**
* Harus tercapai dalam kurun waktu 6 bulan.

# **Analysis**

In [13]:
df.describe()

,Price,Sell Count,Rating
count,1129.000000,1129.000000,1129.000000
mean,22298.445527,369.471213,4.881311
std,24397.007246,1089.674614,0.171910
min,400.000000,1.000000,3.000000
25%,8800.000000,23.000000,4.800000
50%,15000.000000,70.000000,4.900000
75%,25000.000000,250.000000,5.000000
max,210000.000000,10000.000000,5.000000


In [69]:
a = df['Price'].sort_values(ascending=True).to_numpy()
np.quantile(a, 0.25)

8800.0

In [76]:
price_dist = df['Price'].sort_values(ascending=True).to_numpy()
sc_dist = df['Sell Count'].sort_values(ascending=True).to_numpy()
rate_dist = df['Rating'].sort_values(ascending=True).to_numpy()

prc_lst = []
sc_lst = []
rate_lst = []

lists = [prc_lst, sc_lst, rate_lst]
dists = [price_dist, sc_dist, rate_dist]

n = -1

for dst in dists:
    n += 1
    lists[n].append(dst.mean().round(2))                                                            # Mean                     0
    lists[n].append(np.median(dst).round(2))                                                        # Median                   1
    lists[n].append(dst.std().round(2))                                                             # Standard Deviation       2
    lists[n].append(scipy.stats.skew(dst).round(2))                                                 # Skewness                 3
    lists[n].append(scipy.stats.kurtosis(dst).round(2))                                             # Kurtosis                 4
    lists[n].append(np.quantile(dst, 0.25).round(2))                                                # Q1                       5
    lists[n].append(np.quantile(dst, 0.75).round(2))                                                # Q3                       6
    lists[n].append(scipy.stats.iqr(dst).round(2))                                                  # IQR                      7
    lists[n].append(lists[n][6] + (1.5*lists[n][7]))                                                # Upper bound              8
    lists[n].append(lists[n][5] - (1.5*lists[n][7]))                                                # Lower bound              9
    lists[n].append((len(dst[(dst < lists[n][9]) | (dst > lists[n][8])]) / len(dst)).__round__(2))  # Outlier                  10

In [77]:
prc_lst

[22298.45,
 15000.0,
 24386.2,
 3.33,
 14.63,
 8800.0,
 25000.0,
 16200.0,
 49300.0,
 -15500.0,
 0.09]

In [78]:
stats = pd.DataFrame()

stats['Category'] = ['Mean', 'Median', 'Std', 'Skew', 'Kurtosis', 'Q1', 'Q3', 'IQR', 'Upper bound', 'Lower bound', 'Outlier%']
stats['Price'] = lists[0]
stats['Sell Count'] = lists[1]
stats['Rating'] = lists[2]

In [79]:
stats

,Category,Price,Sell Count,Rating
0,Mean,22298.45,369.47,4.88
1,Median,15000.00,70.00,4.90
2,Std,24386.20,1089.19,0.17
3,Skew,3.33,6.33,-4.07
4,Kurtosis,14.63,47.19,29.98
5,Q1,8800.00,23.00,4.80
6,Q3,25000.00,250.00,5.00
7,IQR,16200.00,227.00,0.20
8,Upper bound,49300.00,590.50,5.30
9,Lower bound,-15500.00,-317.50,4.50


## Nomer 1

Dari data yang didapat, dapat diambil beberapa kesimpulan:
* Untuk harga dan jumlah terjual mean > median, maka kedua distribusi tersebut merupakan positively skewed, namun untuk rating median lebih besar daripada mean, maka dari itu distribusi rating merupakan negatively skewed.
* Untuk distribusi harga & jumlah terjual, nilai standard deviasi lebih dari mean, hal ini berarti data dari distribusi harga & jumlah terjual cukup terpencar (spread out).
* Karena semua distribusi memiliki skew diatas |±1|, maka dapat dikategorikan kalau skewness dari ketiga distribusi ini adalah highly skewed.
* Untuk nilai kurtosis ketiga distribusi ini > 0, maka termasuk kedalam leptokurtic.
* Nilai outlier yang paling besar ada di distribusi jumlah terjual di angka 14%, sedangkan untuk harga & rating masing - masing diangka 9% dan 3%

## Nomer 2

In [88]:
min,max = scipy.stats.norm.interval(0.95, loc=price_dist.mean(), scale=prc_lst[2] / np.sqrt(len(price_dist)))

In [93]:
print(f"Min : {min:.2f}")
print(f"Max : {max:.2f}")

Min : 20875.97
Max : 23720.92


Dari sini kita bisa mengetahui potensi minimum pendapatan adalah Rp20.875,97 dan potensi maximum adalah Rp23.720,92

## Nomer 3

**H0 : Harga Jabodetabek = Harga Luar Kota**
**H1 : Harga Jabodetabek != Harga Luar Kota**

In [124]:
jabod = ['Jakarta Pusat', 'Jakarta Barat', 'Jakarta Timur', 'Jakarta Selatan', 'Jakarta Utara', 'Bogor', 'Depok', 'Tangerang', 'Tangerang Selatan', 'Bekasi']

price_jabod = df[df['City'].isin(jabod)]['Price'] # Yang masuk kawasan jabodetabek
price_lukot = df[~df['City'].isin(jabod)]['Price']  # yang masuk kawasan luar kota


In [125]:
print('Retention rate of Price Jabodetabek:',(price_jabod.sum()/len(price_jabod)).__round__(2))
print('Retention rate of Price Luar Kota:',(price_lukot.sum()/len(price_lukot)).__round__(2))

Retention rate of Price Jabodetabek: 23861.11
Retention rate of Price Luar Kota: 21167.6


In [151]:
t_stat,p_val = scipy.stats.ttest_rel(price_jabod.sample(474),price_lukot.sample(474)) # 474 diambil dari length harga jabodetabek
print('P-value:',p_val.round(2)) # print p-value

P-value: 0.2


Karena p-value diatas 0.05, maka H0 (Harga Jabodetabek = Harga Luar Kota) diterima

## Nomer 4

In [156]:
corr_rho, pval_s = scipy.stats.spearmanr(df['Price'], df['Sell Count'])
print(f"rho-correlation: {corr_rho:.2f}, p-value: {pval_s}")

rho-correlation: -0.05, p-value: 0.09850708808908214


Karena p-value > 0.05, maka bisa dibilang bahwa korelasi antara harga dan jumlah terjual adalah kebetulan.

## Konklusi

* Nilai outlier dari dataset yang didapat pada harga, jumlah terjual dan rating adalah masing - masing 9%, 14% dan 3%
* Dari dataset yang sudah di-scrap, distribusi harga dan jumlah terjual memiliki nilai yang terpencar (spread out)
* Sebaiknya penjual seblak menjual seblak di range harga Rp20.875,97 sampai Rp23.720.92
* Harga seblak yang dijual di Jabodetabek sama dengan harga seblak yang dijual di luar kota.
* Meskipun hipotesa awal mengatakan kalau ada kemungkinan bila harga murah maka barang akan terjual dalam jumlah yang banyak, hasil dari uji korelasi mengatakan sebaliknya.